# Segmenting and Clustering Neighbourhoods in Toronto

## 1) Building the Dataframe

Gonna begin by scraping the Wikipedia page using the Beautiful Soup package.
Let's import the libraries:

In [1]:
import pandas as pd
import numpy as np

Now we scrape the table:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)

Hold up, what have we just scraped? How many tables have we got?

In [3]:
print(len(df))

3


Damn that's too many. Let's isolate the right one:

In [4]:
print(df[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

Nice, the first one in the df list is the correct table. Just write that one into its own dataframe:

In [5]:
toronto = df[0]
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


But wait how many boroughs are actually assigned??

In [6]:
toronto['Borough'].replace('Not assigned', np.nan, inplace = True)
toronto['Borough'].count()

103

Let's drop these non-assigned boroughs and check the dimensions:

In [7]:
toronto.dropna(subset = ['Borough'], axis = 0, inplace = True)
toronto.shape

(103, 3)

Are there any NaN neighbourhoods in the dataframe?

In [8]:
toronto['Neighbourhood'].replace('Not assigned', np.nan, inplace = True)
toronto['Neighbourhood'].count()

103

Nope, so we should just be left with assigned data now. Let's sort and check the dataframe:

In [9]:
toronto.sort_values('Postal Code', axis = 0 , inplace = True)
toronto.reset_index(drop=True, inplace=True)
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
toronto.shape

(103, 3)

## 2) Obtaining Spatial Coordinates

Okaay let's try and obtain latitide and longitude coordinates from geocoder:

In [11]:
import geocoder

.\
.\
.\
.\
.\
.\
.\
I tried the method but kept getting timeout messages so I'll roll with the csv:

In [12]:
df_2 = pd.read_csv(r"C:\Users\maxwe\OneDrive\Documents\IBM Exercises\Capstone Project\Geospatial_Coordinates.csv")
df_2.shape

(103, 3)

In [13]:
df_2.sort_values('Postal Code', axis = 0 , inplace = True)
df_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sick, looks like we've got the same postcodes as in the other dataframe. Let's concatenate them:

In [14]:
coords = df_2.drop(columns = 'Postal Code')
coords.reset_index(drop=True, inplace=True)
df_loc = pd.concat([toronto, coords], axis = 1)
df_loc

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## 3) Explore and Cluster the Neighbourhoods

Alright. I'm gonna attempt a similar analysis to the New York one. First let's get some libraries imported:

In [15]:
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

### 3a) Check location data and create initial visualisations

I'll define an instance of the geocoder called 'toronto_exp' and obtain the coordinates according to GeoPy:

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Let's get an interactive map going:

In [17]:
# Create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df_loc['Latitude'], df_loc['Longitude'], df_loc['Borough'], df_loc['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Niiice, happy with this. Let's narrow it down a bit though - what are the different boroughs in Toronto?

In [18]:
df_loc.groupby(['Borough']).count()

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


We'll work with the first three boroughs in the table above:

In [19]:
toronto_data = df_loc[df_loc['Borough'] <= 'East Toronto'].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(33, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let's visualise this subset of neighbourhoods:

In [20]:
# Create map of Toronto using latitude and longitude values
toronto_map_2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], 
                                            toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map_2)  
    
toronto_map_2

Sound. Let's call Foursquare API and explore these neighbourhoods more.

### 3b) Explore the Neighbourhoods

In [21]:
CLIENT_ID = 'IHQTVX3GAMMK5TM3S4BAX22RRQKPP4J2PTSDWNRXXRO2VUEX' # your Foursquare ID
CLIENT_SECRET = 'XVIYOKVGJUH2XFZYSARL2PMNEHP0VRA2Z2KKXEI5QGGJVBWP' # your Foursquare Secret
VERSION = '20201116' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

We want a function which perform the following for each of the neighbourhoods in `toronto_data`:

1. Create a GET request for nearby venues and send it to the API
2. Extract the category of venues using the `get_category_type` function  
3. Read the json into a dataframe.

We'll use an adaption of the `getNearbyVenues` from the New York exercise:

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Nbh Latitude', 
                  'Nbh Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now that we've got our function, let's input our `toronto_data` and write it into a dataframe:

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
Business reply mail Processing Centre, South Central Letter 

What have we got??

In [24]:
toronto_venues

,Neighbourhood,Nbh Latitude,Nbh Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
...,...,...,...,...,...,...,...
1466,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1467,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1468,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1469,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


Wicked.

### 3c) Analyse the Neighbourhoods

Let's analyse based on most common venue types. To do this, we:

1. Add dummy variables for each of the venue categories
2. Group rows by neighbourhood and calculate the mean frequency of occurence for each venue category
3. Write this information into a pandas dataframe

In [25]:
# Step 1

# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [26]:
# Step 2

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.000000
1,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.0,0.014706
4,Christie,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [27]:
# Step 3

# Write a function to sort the venues in descening order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# Create a dataframe and display the top 10 venue categories for each neighbourhood

# Initialise parameters
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Concert Hall,Liquor Store
1,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Comic Shop,Park,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Boutique,Airport Terminal
3,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant
4,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Athletics & Sports,Restaurant,Italian Restaurant,Baby Store,Nightclub


Pretty sweet.

### 3d) Cluster the Neighbourhoods

Let's run K-means clustering on the sorted venues data to try and learn something about the neighbourhoods. We'll set it with 3 clusters (pretty arbitrarily)

In [29]:
# Set number of clusters
k = 3

# Get our dataframe ready by removing the Neighbourhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

We can check which clusters our neighbourhoods fall into by adding a label column to our dataframe:

In [31]:
# Add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
toronto_merged = toronto_data

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Indian Restaurant,Spa,Caribbean Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Sushi Restaurant,Pub,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [34]:
toronto_merged.groupby(['Cluster Labels']).count()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


Now we can visuliase the clusters:

In [35]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3e) Examine the Clusters

So 28 of our neighbourhoods fall into the first cluster, 4 into the second and 1 into the third depicted in mint green.
Let's quickly check Roselawn to see if there's anything about its venues compared to a sample of first cluster neighbourhoods which make it unique:

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,2,Music Venue,Garden,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [37]:
cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster_0

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Indian Restaurant,Spa,Caribbean Restaurant
2,"India Bazaar, The Beaches West",0,Park,Sushi Restaurant,Pub,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,Studio District,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood
5,Davisville North,0,Gym / Fitness Center,Sandwich Place,Department Store,Dog Run,Hotel,Dance Studio,Food & Drink Shop,Breakfast Spot,Park,Donut Shop
6,"North Toronto West, Lawrence Park",0,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
7,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Italian Restaurant,Gym,Coffee Shop,Sushi Restaurant,Park,Indian Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",0,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Bank,Restaurant,Pizza Place,Liquor Store
11,"St. James Town, Cabbagetown",0,Coffee Shop,Restaurant,Café,Pizza Place,Pub,Italian Restaurant,Park,Chinese Restaurant,Market,Bakery
12,Church and Wellesley,0,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Restaurant,Café,Hotel,Mediterranean Restaurant,Yoga Studio,Men's Store


In [38]:
cluster_0.groupby(['1st Most Common Venue']).size().sort_values(ascending = False)

1st Most Common Venue
Coffee Shop             18
Sandwich Place           2
Gym / Fitness Center     2
Pub                      1
Park                     1
Grocery Store            1
Greek Restaurant         1
Café                     1
Airport Lounge           1
dtype: int64

In [39]:
cluster_0.groupby(['2nd Most Common Venue']).size().sort_values(ascending = False)

2nd Most Common Venue
Café                   6
Clothing Store         2
Restaurant             2
American Restaurant    2
Sushi Restaurant       1
Aquarium               1
Auto Workshop          1
Bakery                 1
Bank                   1
Bookstore              1
Coffee Shop            1
Sandwich Place         1
Dessert Shop           1
Gay Bar                1
Health Food Store      1
Hotel                  1
Italian Restaurant     1
Mexican Restaurant     1
Pub                    1
Airport Service        1
dtype: int64

Just from brief observation, we can see that Coffee Shops and Cafes are the most frequently occuring venues in cluster 0 neighbourhoods. This may go some way to explaining why they were clustered together.

In contrast, Roselawn in cluster 2 has nearby venue categories not seen at all in cluster 0, such as Ethiopian Restaurant(s) and Escape Room(s).



**This concludes this notebook. Thanks for reading!!**